In [12]:
%pip install --user boto3

  Using cached boto3-1.17.21-py2.py3-none-any.whl (130 kB)
  Using cached jmespath-0.10.0-py2.py3-none-any.whl (24 kB)
  Using cached botocore-1.20.21-py2.py3-none-any.whl (7.3 MB)
  Using cached s3transfer-0.3.4-py2.py3-none-any.whl (69 kB)
Note: you may need to restart the kernel to use updated packages.


In [14]:
import boto3

In [17]:
s3 = boto3.resource('s3',
                   aws_access_key_id='',
                   aws_secret_access_key='')

In [19]:
try:
    s3.create_bucket(Bucket='datacont-grey', CreateBucketConfiguration={'LocationConstraint':'us-east-2'})
except:
    print("This may already exist")

This may already exist


In [20]:
bucket = s3.Bucket("datacont-grey")

In [21]:
bucket.Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'Q3ETCVD94SPPYNWR',
  'HostId': 'Cx7UrMIuJFCwmRv5HCGTj1ENRkI9y9wwTfAvPbqxLrHAaUAAFdPfnPNOwamzlN4UHdnEYvnjND0=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Cx7UrMIuJFCwmRv5HCGTj1ENRkI9y9wwTfAvPbqxLrHAaUAAFdPfnPNOwamzlN4UHdnEYvnjND0=',
   'x-amz-request-id': 'Q3ETCVD94SPPYNWR',
   'date': 'Thu, 04 Mar 2021 23:18:59 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [22]:
body = open('./exp1.csv', 'rb')

In [23]:
o = s3.Object('datacont-grey', 'exp1').put(Body=body)

In [34]:
s3.Object('datacont-grey', 'exp1').Acl().put(ACL='public-read')

{'ResponseMetadata': {'RequestId': 'B51HG3NCK1535D4W',
  'HostId': 'MmD52MGqE9DnPQKT2mvIYbZSqjR/8l51u2Wzf6C4UTagMJ0IpLu7BBlCoxDqU/3C46ZM1CaGNdE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'MmD52MGqE9DnPQKT2mvIYbZSqjR/8l51u2Wzf6C4UTagMJ0IpLu7BBlCoxDqU/3C46ZM1CaGNdE=',
   'x-amz-request-id': 'B51HG3NCK1535D4W',
   'date': 'Thu, 04 Mar 2021 23:45:05 GMT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [27]:
dyndb = boto3.resource('dynamodb', region_name='us-east-2',
                     aws_access_key_id='',
                     aws_secret_access_key='')

In [29]:
table = dyndb.create_table(
    TableName='DataTable',
    KeySchema=[
        {'AttributeName': 'PartitionKey', 'KeyType': 'HASH'},
        {'AttributeName': 'RowKey', 'KeyType': 'RANGE'}
    ],
    AttributeDefinitions=[
        {'AttributeName': 'PartitionKey', 'AttributeType': 'S'},
        {'AttributeName': 'RowKey',       'AttributeType': 'S'}
    ],
    ProvisionedThroughput={
        'ReadCapacityUnits': 5,
        'WriteCapacityUnits': 5
    }
)

In [31]:
table.meta.client.get_waiter('table_exists').wait(TableName='DataTable')

In [32]:
print(table.item_count)

0


In [33]:
import csv

In [35]:
body = open('test.jpg', 'rb')

In [44]:
urlbase = 'https://s3-us-east-2.amazonaws.com/datacont-grey'
with open('experiments2.csv') as csvfile:
    csvf = csv.reader(csvfile, delimiter=',', quotechar='|')
    for i,item in enumerate(csvf):
        if i > 0:
            body = open('./'+item[4], 'rb')
            s3.Object('datacont-grey', item[4]).put(Body=body)
            md = s3.Object('datacont-grey', item[4]).Acl().put(ACL='public-read')
            url = urlbase + item[3]
            metadata_item={'PartitionKey': item[0], 'RowKey': item[1], 'description': item[3], 'date': item[2], 'url':url}
            table.put_item(Item=metadata_item)